# Query Cell Types Ontology

# Context

This notebook has been put together for the MMB demo on 2022-05-30: see [slides](https://docs.google.com/presentation/d/1Ib1_8byK0hVuNS-wPbqmeL5Lcf67m_oegC3-czPw5ws/edit#slide=id.g116ba5ed71e_0_8)

## Imports

In [ ]:
import json
import rdflib
import getpass
import pandas as pd
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
from bmo_tools.ontologies import subontology_from_term
import pprint
from kgforge.core import KnowledgeGraphForge

## Setup

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN,
                            endpoint="https://staging.nise.bbp.epfl.ch/nexus/v1",
                            bucket="neurosciencegraph/datamodels")

## Ontologies

### Set brain region

In [ ]:
BRAIN_REGION = "Cerebral cortex"
# BRAIN_REGION = "Cerebellum"

Get brain region id

In [ ]:
r = forge.search({"label": BRAIN_REGION})

In [ ]:
brain_region = r[0].id

In [ ]:
brain_region

## Queries

### Which brain regions do have neuron t-types available

In [ ]:
query = f"""

SELECT ?brain_region

WHERE {{
        ?t_type_id subClassOf* <https://bbp.epfl.ch/ontologies/core/celltypes/NeuronTranscriptomicType> ;
                  canHaveBrainRegion ?brain_region_id .        
        ?brain_region_id label ?brain_region .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=1000)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
set(df.brain_region)

### Get possible t-types for a given brain region

In [ ]:
query = f"""

SELECT ?brain_region ?t_type

WHERE {{
        ?t_type_id label ?t_type ;
            canHaveBrainRegion <{brain_region}> .        
        <{brain_region}> label ?brain_region . 
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head()

### Get possible met-type combinations plus excitatory/inhibitory category for a given brain region

In [ ]:
query = f"""

SELECT ?brain_region ?transmitter ?t_type ?m_type ?e_type

WHERE {{
        ?t_type_id label ?t_type ;
                canHaveBrainRegion <{brain_region}> .        
        <{brain_region}> label ?brain_region .
        
        ?m_type_id label ?m_type ;
            subClassOf* MType ;
            canHaveTType ?t_type_id ;
            subClassOf* / hasNeurotransmitterType / label ?transmitter .

        ?e_type_id label ?e_type ;
            subClassOf* EType ;
            subClassOf* / canHaveMType ?m_type_id ;
            subClassOf* / canHaveTType ?t_type_id .            
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head()

### Get m-types with a basket cell morphology

In [ ]:
query = f"""

SELECT ?basket_cell

WHERE {{
        ?basket_cell_id subClassOf+ / hasMorphologicalPhenotype / label "basket cell morphology" ;
                  label ?basket_cell .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df.head(20)

### Get definition of a basket cell morphology

In [ ]:
query = f"""

SELECT ?basket_cell ?definition

WHERE {{
        ?basket_cell_id subClassOf+ / hasMorphologicalPhenotype ?pato_id ;
            label ?basket_cell .
        ?pato_id label "basket cell morphology" ;
                <http://purl.obolibrary.org/obo/IAO_0000115> ?definition .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=20000)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
set(df.definition)

### Get t-types from Yao et al. 2021 paper

In [ ]:
query = f"""

SELECT ?label

WHERE {{
        ?id seeAlso <https://www.sciencedirect.com/science/article/pii/S0092867421005018?dgcid=rss_sd_all> ;
            label ?label .
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df

# M-types classed by transmitter type

In [ ]:
query = f"""

SELECT ?transmitter ?m_type

WHERE {{
        ?m_type_id label ?m_type ;
            subClassOf* MType ;
            subClassOf* / hasNeurotransmitterType / label ?transmitter .           
}}
"""

In [ ]:
resources = forge.sparql(query, limit=100)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df

# Placeholders

In [ ]:
query = """

SELECT ?id ?label

WHERE {
        ?id label ?label .
        FILTER (CONTAINS(STR(?label), 'Placeholder'))
}
"""

In [ ]:
resources = forge.sparql(query, limit=100)

In [ ]:
df = forge.as_dataframe(resources)

In [ ]:
df